In [ ]:
def train_sent(read_f, write_f, length):
    with open(read_f, 'r') as f:
        count = 0
        for sent in f:
            if count >= length:
                break
            with open(write_f, 'a+') as w:
                if len(sent) > 1:
                    w.write(sent)
                    count += 1

In [ ]:
def txt_write_csv(csv_file, txt_file):
    import pandas as pd
    sent = []
    label = []
    with open(txt_file, 'r') as f:
        for line in f:
            sent.append(line)
            label.append('n')
    dataframe = pd.DataFrame({'sentence':sent, 'label':label})
    dataframe.to_csv(csv_file, index=False, sep=',')

In [1]:
import pandas as pd
import numpy as np
from gensim.models import word2vec
def sent_vector(csv_file, length):
    if length > 0:
        csv_data = pd.read_csv(csv_file, low_memory = False, nrows=length)#防止弹出警告
    else:
        csv_data = pd.read_csv(csv_file, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    sentences = csv_df['sentence']
    label = csv_df['label']
    label.loc[label == 'n'] = 0
    label.loc[label == 'p'] = 1
    model = word2vec.Word2Vec.load('Model/n1_cbow.w2v')
    size = model.wv.syn0.shape[1]
    sents_vec = []
    for sent in sentences:
        sents_vec.append(compute_sent_vec(sent, model, size))
    return np.array(sents_vec), label, sentences
        

In [6]:
def compute_sent_vec(sent, model, size):
    vec = np.zeros(size)
    for word in sent.split():
        if word in model.wv:
            vec += model.wv[word]
    vec /= len(sent.split())
    return vec


In [7]:
sents_vec, label, sentences = sent_vector('Data/training_data.csv', -1)

/workspace/anaconda3/envs/acai/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  from ipykernel import kernelapp as app


In [ ]:
#self implemented logistic regression
from classifier.logistic_regression import LogisticRegression
l = LogisticRegression(sents_vec[:700], label[:700])
cost = l.train(0.1,1000)
pred_y = l.predict(sents_vec[300:], label[300:])

In [ ]:
#sklearn logistic regression result
from sklearn.linear_model import LogisticRegression
label = np.array(label, dtype='int')
clf = LogisticRegression(max_iter=1000).fit(sents_vec[:700], label[:700])
test_accuracy = clf.score(sents_vec[300:], label[300:])*100
pred = clf.predict(sents_vec[300:])
print("正确率为%s%%" % test_accuracy)

In [8]:
# self implemented k means
from cluster.k_means import kMeans
from collections import defaultdict
km = kMeans()
km.train(3, sents_vec[:1000])
result = km.predict(sents_vec[:1000])
sents_cluster = defaultdict(list)
x = [sents_cluster[result[i]].append(sentences[i]) for i in range(len(result))]

In [12]:
sents_cluster[1]

['欢迎 各位 袋友 到此一游\n',
 '午安\n',
 '跨跨 跨\n',
 '跨 一下\n',
 '可怕 再 跨\n',
 '自觉 跨 一个 🔥\n',
 '跨 火盆\n',
 '....\n',
 '肚子饿\n',
 '弱国 无 外交 … …\n',
 '— —\n',
 '跨 🔥\n',
 '跨 🔥\n',
 '… …\n',
 '看到 ps 痕迹 。 我 …\n',
 '… …\n',
 '… …\n',
 '… …\n',
 'i   like   it\n',
 '踢 一脚\n',
 'https : m . weibo . cn52921076944127663834677764 肉 饱饱\n',
 '.....\n',
 '… …\n',
 '… …\n',
 '… …\n',
 '… …\n',
 '… …\n',
 '… …\n',
 '… …\n',
 '… …\n',
 '北极星\n',
 '围观 ！\n',
 '戛纳\n',
 '… …\n',
 '… …\n',
 '- - - - - -\n',
 '老葛 今天 真的 突破 颜值 下限 了 … …\n',
 'zzf 这个 发型 … …\n',
 '… …\n',
 '嘴 … …\n',
 '可怜 的 gg … …\n',
 '____ （ （ （ （ （\n',
 '____ （ （ （ （ （\n',
 '- - -\n',
 '____ （ （ （ （ （\n',
 '脱饭 1128 秒\n',
 '… …\n',
 '史诗 级 吊粉\n',
 '… …\n',
 'you   get   it 等艳玲\n',
 '- - -\n']

In [ ]:
from sklearn.cluster import KMeans
from collections import defaultdict
kmeans = KMeans(n_clusters=3).fit(sents_vec[:1000])
result = kmeans.predict(sents_vec[:1000])
sents_cluster1 = defaultdict(list)
x = [sents_cluster1[result[i]].append(sentences[i]) for i in range(len(result))]

In [ ]:
sents_cluster1[1]

In [13]:
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
iris=load_iris()
iris_data=iris['data']
iris_target=iris['target']
data_zs=(iris_data-iris_data.mean())/iris_data.std()
## 也可以自定义函数minmax标准化、或者现成的函数
scale=MinMaxScaler().fit(iris_data)
iris_datascale=scale.transform(iris_data)
kmeans=KMeans(n_clusters=3).fit(iris_datascale)
result=kmeans.predict(iris_datascale)

In [14]:
result

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2,
       2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0], dtype=int32)

In [15]:
from sklearn.datasets import load_iris
from cluster.k_means import KMeans
from sklearn.preprocessing import MinMaxScaler
iris=load_iris()
iris_data=iris['data']
iris_target=iris['target']
data_zs=(iris_data-iris_data.mean())/iris_data.std()
## 也可以自定义函数minmax标准化、或者现成的函数
scale=MinMaxScaler().fit(iris_data)
iris_datascale=scale.transform(iris_data)
km = KMeans()
km.train(3, iris_datascale)
result1=km.predict(iris_datascale)

In [16]:
np.array(result1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1,
       2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1,
       1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1,
       1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1,
       2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2])

In [17]:
iris_datascale

array([[0.22222222, 0.625     , 0.06779661, 0.04166667],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.11111111, 0.5       , 0.05084746, 0.04166667],
       [0.08333333, 0.45833333, 0.08474576, 0.04166667],
       [0.19444444, 0.66666667, 0.06779661, 0.04166667],
       [0.30555556, 0.79166667, 0.11864407, 0.125     ],
       [0.08333333, 0.58333333, 0.06779661, 0.08333333],
       [0.19444444, 0.58333333, 0.08474576, 0.04166667],
       [0.02777778, 0.375     , 0.06779661, 0.04166667],
       [0.16666667, 0.45833333, 0.08474576, 0.        ],
       [0.30555556, 0.70833333, 0.08474576, 0.04166667],
       [0.13888889, 0.58333333, 0.10169492, 0.04166667],
       [0.13888889, 0.41666667, 0.06779661, 0.        ],
       [0.        , 0.41666667, 0.01694915, 0.        ],
       [0.41666667, 0.83333333, 0.03389831, 0.04166667],
       [0.38888889, 1.        , 0.08474576, 0.125     ],
       [0.30555556, 0.79166667, 0.05084746, 0.125     ],
       [0.22222222, 0.625     ,

In [19]:
sents_vec[0:2]

array([[-1.09014413e+00,  3.48017942e-01,  3.61643959e-02,
         1.88938348e-01,  1.99290381e-01,  3.14928913e-01,
         9.01278453e-01,  7.18106316e-01,  7.96017296e-01,
        -7.49295274e-01, -2.44985569e-01, -2.44651151e-01,
         3.53359180e-02, -4.87918431e-01, -1.54740796e-01,
        -1.27185880e-01, -7.65349819e-01,  3.12628812e-01,
        -7.55293672e-02, -9.47086302e-01, -1.14877573e+00,
        -7.04916845e-01,  1.39734466e-01, -1.13291339e+00,
        -4.68124491e-01,  4.11575178e-02,  1.71918118e-01,
        -6.41566588e-01,  1.15523661e+00, -5.47202584e-01,
         5.37963121e-01, -8.12233704e-01,  8.90096237e-01,
        -7.65963740e-01, -3.35747347e-01, -1.53000980e-01,
         1.13677073e-03, -9.31074972e-03,  4.46866470e-01,
         7.33246661e-01,  1.75139692e-01,  4.96492926e-01,
         7.79976089e-01,  5.75778521e-01, -6.00243829e-02,
        -8.27554998e-01, -2.17352627e-01, -1.40406107e-01,
        -4.75978727e-01,  1.89328452e-01,  4.36652346e-0